In [ ]:
import xmlrpc.client
import pymssql
import schedule
import time
from datetime import datetime

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# MS SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'  # Replace with your SQL server name or IP
database_name = 'Odoo_sql_database'   # Replace with your target database name
sql_user_name = 'SuperAdmin'   # Replace with your SQL server username
sql_password = 'SuperAdmin'    # Replace with your SQL server password

def sync_sales_report():
    print("Starting synchronization of sales report...")

    # Connect to Odoo
    common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common')
    uid = common.authenticate(db_name, username, password, {})

    if not uid:
        print("Failed to authenticate. Check your credentials.")
        return

    print(f"Authenticated successfully. User ID: {uid}")

    # Define the 'models' object
    models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object')

    # Define the start and end date for January 2024
    start_date = '2025-01-01'
    end_date = '2025-01-31'

    # Correct the date filter in the domain
    domain = [('date', '>=', start_date), ('date', '<=', end_date)]

    # Define the fields to retrieve
    fields = [
        'name', 'date', 'product_id', 'product_uom', 'product_uom_qty', 
        'qty_delivered', 'qty_to_invoice', 'qty_invoiced', 'partner_id', 
        'company_id', 'user_id', 'price_total', 'price_subtotal', 
        'untaxed_amount_to_invoice', 'untaxed_amount_invoiced', 'product_tmpl_id', 
        'categ_id', 'nbr', 'pricelist_id', 'analytic_account_id', 'team_id', 
        'country_id', 'industry_id', 'commercial_partner_id', 'weight', 
        'volume', 'discount', 'discount_amount', 'campaign_id', 'medium_id', 
        'source_id', 'order_id', 'warehouse_id', 'state', 'id', 'display_name', 
        '__last_update'
    ]

    # Fetch the sales report data from Odoo
    try:
        report_data = models.execute_kw(
            db_name,
            uid,
            password,
            'sale.report',  # Model name (adjust if needed)
            'search_read',  # Operation to read the data
            [domain, fields],  # Correct domain filter and field list
            {'limit': 1000}  # Optional: limit the number of records
        )

        if not report_data:
            print("No data fetched from Odoo. Exiting synchronization.")
            return

        print(f"Fetched {len(report_data)} rows of sales report data.")

        # Insert the sales data into MS SQL Server
        try:
            # Connect to SQL Server
            conn = pymssql.connect(server=server_name, user=sql_user_name, password=sql_password, database=database_name)
            cursor = conn.cursor()

            # Loop through the data and insert row-wise
            for record in report_data:
                # Flatten the nested list values
                flat_record = {}
                for field, value in record.items():
                    if isinstance(value, list) and len(value) == 2:  # This handles fields like product_id, partner_id, etc.
                        flat_record[f"{field}_id"] = value[0]  # ID part
                        flat_record[f"{field}_name"] = value[1]  # Name part
                    else:
                        flat_record[field] = value

                # Ensure column names match the fields in the table
                columns = ', '.join(flat_record.keys())
                placeholders = ', '.join(['%s'] * len(flat_record))
                values = list(flat_record.values())

                # Construct and execute the SQL insert query
                insert_query = f"""
                INSERT INTO sale_report ({columns})
                VALUES ({placeholders})
                """
                cursor.execute(insert_query, values)

            # Commit the transaction
            conn.commit()
            print(f"Inserted {len(report_data)} rows of sales report data into the SQL Server database.")

            # Close the connection
            cursor.close()
            conn.close()

        except pymssql.Error as e:
            print(f"Error while connecting to SQL Server or executing query: {e}")

    except xmlrpc.client.Fault as e:
        print(f"Error fetching data from Odoo: {e}")

# Schedule the function to run every 4 hours
schedule.every(4).hours.do(sync_sales_report)  # Correct function name

# Run the function immediately
sync_sales_report()  # Correct function name

print("Scheduler started. The script will also run every 4 hours.")

# Keep the script running for scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(1)


Starting synchronization of sales report...
Authenticated successfully. User ID: 35
Fetched 1000 rows of sales report data.
Inserted 1000 rows of sales report data into the SQL Server database.
Scheduler started. The script will also run every 4 hours.
